>>> Work in progress

### Graph as Matrix
- determine the node importance using random walk - PageRank
- obtain node embeddings in the form of matrix factorization
- random walk, MF and node embeddings relationship

### PageRank
- Web as a graph
  - Node - web page
  - Edge - hyperlink
  - lot of static pages (navigational)
  - now-a-days there are lot of transactional pages
- represent as directed graph
- what page on web is more important than other

#### Link Analysis
- compute importanceo of nodes in graph
  - PageRank
  - Personalized Page rank
  - Random walk with restarts
- all in-links are equal?
  - recursive question
- the flow model
  - in-links
  - out-links
  - are all in-links equal
    - links from important pages count more

##### The flow model
- it works on the vote model
- a vote from important page has higher importance
- a page is important if it is pointed to by other important pages
- if page $i$ with importance $r_{i}$ has $d_{i}$ out-links, each link gets $r_{i}/d_{i}$ votes
  - For example, node k has 4 out links, so it shares it as $r_{k}/4$
> rank $r_{j} = \sum\limits_{i \rightarrow j} \frac{r_{i}}{d_{i}}$  
> $d_{i}$ - out-degree of node i

<img src="./images/04_pageRank_FlowModel.png" width=200 height=200>  
$\tiny{\text{YouTube-Stanford-CS224W-Jure Leskovec}}$

- solving this with Gaussian elimination is not a scalable problem

<img src="./images/04_pageRank_FlowModel3.png" width=200 height=200>  
$\tiny{\text{YouTube-Stanford-CS224W-Jure Leskovec}}$

##### Stochastic adjacency matrix
- If j points to i ($j \rightarrow i$), then $M_{ij} = \frac{1}{d_{j}}$
- M is a column stochastic matrix
- cols sum to 1
- prob distribution over matrix M
- rank vector - $\sum_{i}r_{i} = 1$
- the flow equation can be written as 
> $r = M.r$  
> or, $r_{j} = \sum\limits_{i \rightarrow j} \frac{r_{i}}{d_{\text{out}-i}}$ 

<img src="./images/04_pageRank_Matrix.png" width=200 height=200>  
$\tiny{\text{YouTube-Stanford-CS224W-Jure Leskovec}}$

#### Connection to Random walk
- random web surfer
- at time t, he is on page i
- at time t+1, he follows an out-link from i _uniformly at random_
- and ends up at page j
- repeats the process indefinitely
- p(t) vector in which the coordinate prob that he is at page i at time t
- p(t) is the prob distribution over pages
- where is the surfer at time t+1
> p(t+1) = M.p(t)
- p(t) is the prob vector where the random walker was at prev time step
- p(t+1) is the prob distrib/vector of where the walker will be at the next time step


#### The stationary distribution
- suppose the random walk reaches a steady state at time t
- p(t) is a stationary distribution of random walk
> p(t+1) = M.p(t) = p(t)
- so the original rank vector r is a stationary distribution for the random walk

#### Eigenvector of matrix
- Adjacency matrix satisfies $\lambda c = A c$
- c: eigenvector, $\lambda$: eigenvalue

#### Eigenvector formulation
- Power iteration
    - starting from any vector u, the limit M(M(M...(M(Mu))) is the long-term distribution of surfer
    - r is the principal eigenvector of M with eigenvalue 1
    - the flow equation 1.r = Mr

### How to solve PageRank
- given a graph with n nodes, use iterative procedure
  - randomly assign each node an initial page rank
  - repeat until the vector r stabilizes/converge $\sum_{i}|r_{i}^{t+1} - r_{i}^{t}| < \epsilon$
    - calculate page rank of each node
    > $r_{j}^{(t+1)} = \sum\limits_{i \rightarrow j} \frac{r_{i}^{(t)}}{d_{\text{out}-i}}$ 
- generally speaking, it takes 50 iterations to reach stationary distribution or limiting solution
- Google runs this algorithm every day on the entire web


#### Dead-ends and Spider-traps
- Adding random uniform teleportation solves issues of dead-ends(no out-links) and spider-traps(all out-links are within the group)
- Spider traps
  - with prob $\beta$, follow a link at random
  - with prob $1-\beta$, jump to a random page
  - $\beta$ generally is in the range of 0.8 to 0.9
- Dead-ends
  - random teleport links with total prob of 1

<img src="./images/04_pageRank_DeadEnd.png" width=200 height=200>  
$\tiny{\text{YouTube-Stanford-CS224W-Jure Leskovec}}$

#### PageRank equation
> $r_{j} = \sum\limits_{i \rightarrow j} \beta\frac{r_{i}}{d_{i}} + (1-\beta)\frac{1}{N}$ - flow based formulation   
> $G = \beta M + (1-\beta)[\frac{1}{N}]_{NxN}$ - matrix equation 
> $r = G.r$ - recursive problem  
- this formulation assumes that M has no dead ends, either preprocess matrix M to remove all dead ends or explicitly follow random teleport links with prob 1.0 from dead ends

### Random walk with restarts and personalized PageRank
- suppose there is a bipartite graph representing user and item interactions
- for example recommendation from Amazon, based on the user purchase items
- the next question that occurs is, based on the past order history, what item will he purchase next?
- or based on items Q and P sell history from similar users, will user recommend item P when user interacts with Q
- how to define the concept of proximity or relatedness of different items in this graph?
- which is more related A,A' or B,B'

<img src="./images/04_bipartiteUserItemGraph1.png" width=400 height=400>  
$\tiny{\text{YouTube-Stanford-CS224W-Jure Leskovec}}$

#### Node proximity measures
- If we look at A,A', B,B' only
  - A has a shorter path than B and B', so A and A' are more related
- but if we look at C and C' which has 2 users, say both purchased these 2 items, then C and C' are more related as they have common neighbor
- We need to design algorithm/metric that considers the shortest path but also considers the common neighbor and how many different paths allow you to go from one to another


<img src="./images/04_bipartiteNodeProximity1.png" width=400 height=400>  
$\tiny{\text{YouTube-Stanford-CS224W-Jure Leskovec}}$

- PageRank can help to solve this
- suppose another new user has purchased item D and D'
- but has enjoyed buying lot of other items
- so the relationship is less strong than users of item C and C'

<img src="./images/04_bipartiteNodeProximity2.png" width=400 height=200>  
$\tiny{\text{YouTube-Stanford-CS224W-Jure Leskovec}}$

#### Proximity on graphs

##### PageRank
- how does the notion of PageRank gets extended here
- PageRank tells me the importance of node on graph and ranks nodes by importance
- it has a notion of teleport where a random surfer teleports uniformly over any node in graph
- S = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]

##### Personalized PageRank
- in this extension of PageRank, when it teleports, it does not teleport to anywhere, instead it teleports to a subset of nodes S
- S = [0.1, 0, 0, 0.2, 0, 0, 0.5, 0, 0, 0.2]

##### Proximity on graphs
- when it teleports, if it teleports to a single node S, which is the starting node
- this is called _Random Walk with Restart_
- S = [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]


#### Random walk with Restart - Algorithm
- Query Nodes Q
- Proximity to query node
- Number of visits by random walk starting at Q

### Matrix Factorization and Node Embeddings
- two nodes are similar if they coappear in the same random walk, starting at a given node

#### Node similarity
- factorize adjacency matrix as a product of $Z$ and $Z^{T}$
> $Z^{T}Z = A$

#### Matrix Factorization
- the embedding matrix dimension d (number of rows) is much smaller than number of nodes n
- so exact factorization $A = Z^{T}Z$ is not possible
- to learn Z approximately
  - find the matrix Z such that
  > min$_{Z}\Vert A - Z^{T}Z\Vert_{2}$
  - use L2 norm or softmax function
- The inner product decoder with node similarity defined by edge connectvity is equivalent to matrix factorization of adjacency matrix A
- approximate A by the embeddings of the nodes such that if two nodes are linked then the dot product should be equal to 1 otherwise 0

#### Random walk based similarity
- Deepwalk and node2vec have more complex node similarity based on random walks
- Deepwalk Equation  

<img src="./images/04_matrixFac_DeepwalkEqn.png" width=400 height=200>  
$\tiny{\text{YouTube-Stanford-CS224W-Jure Leskovec}}$

##### Limitations 1
- if new nodes are appearing over time, then the nodes that are not in the embedding computations and will not be calculated

##### Limitations 2
- structural similarity - even if there are two different graphs, their local network structure is very similar 
  - node2vec and deepwalk will come up with very different embedding for these structural similar network
  - it wont be able to compute structural similarity
  - but it will be able to capture the identities of neighbors next to a given starting node
  - anonymous walks will be able to capture the structural similarity

##### Limitations 3
- this approach cannot utilize node edge in graph level features, i.e., feature vector attached to nodes, edges and graphs do not work in this framework
  - the solution is deep representation learning and graph neural network
